In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [2]:
df = pd.read_csv('dfrandom.csv')
df.head()

/Users/ADMIN/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,customer_id,108775015,108775044,108775051,110065001,110065002,110065011,111565001,111586001,111593001,...,946795001,946827001,947060001,947509001,949198001,949551001,949551002,952938001,953763001,All
0,86,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
1,173,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8
2,733,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,118
3,794,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
4,956,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,142


In [3]:
df = df.set_index('customer_id').drop(columns='All',index='All')

In [5]:
article = pd.read_csv('articles.csv')
article.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

#### Model

In [57]:
from sklearn.metrics.pairwise import cosine_similarity
import operator

def similar_users(user_id, matrix, k=3):
    # create a df of just the current user
    user = matrix[matrix.index == user_id]
    
    # and a df of all other users
    other_users = matrix[matrix.index != user_id]
    
    # calc cosine similarity between user and each other user
    similarities = cosine_similarity(user,other_users)[0].tolist()
    
    # create list of indices of these users
    indices = other_users.index.tolist()
    
    # create key/values pairs of user index and their similarity
    index_similarity = dict(zip(indices, similarities))
    
    # sort by similarity
    index_similarity_sorted = sorted(index_similarity.items(), key=operator.itemgetter(1))
    index_similarity_sorted.reverse()
    
    # grab k users off the top
    top_users_similarities = index_similarity_sorted[:k]
    users = [u[0] for u in top_users_similarities]
    
    return users

In [80]:
from tqdm import tqdm

USER = []
for i in  tqdm(range(0,10)):
#     n = get_recommendations(name[i]).tolist()
    similar_user_indices = similar_users(df.index[i], df)
    USER.append(similar_user_indices)
    
newdf = pd.DataFrame(list(zip(df.index, USER)),columns =['article_id_last', 'similarid'])
newdf.head()

100%|██████████| 10/10 [01:25<00:00,  8.57s/it]


,article_id_last,similarid
0,86,"[137191, 936656, 697619]"
1,173,"[708024, 392783, 646417]"
2,733,"[795567, 569413, 1303233]"
3,794,"[655329, 358194, 367018]"
4,956,"[460516, 988067, 1309055]"


In [81]:
def recommend_item(user_index, similar_user_indices, matrix, items=12):
    
    # load vectors for similar users
    similar_users = matrix[matrix.index.isin(similar_user_indices)]
    # calc avg ratings across the 3 similar users
    similar_users = similar_users.mean(axis=0)
    # convert to dataframe so its easy to sort and filter
    similar_users_df = pd.DataFrame(similar_users, columns=['mean'])
    
    
    # load vector for the current user
    user_df = matrix[matrix.index == user_index]
    # transpose it so its easier to filter
    user_df_transposed = user_df.transpose()
    # rename the column as 'rating'
    user_df_transposed.columns = ['count']
    # remove any rows without a 0 value. Anime not watched yet
    user_df_transposed = user_df_transposed[user_df_transposed['count']==0]
    # generate a list of animes the user has not seen
    items_notbuy = user_df_transposed.index.tolist()
    
    # filter avg ratings of similar users for only anime the current user has not seen
    similar_users_df_filtered = similar_users_df[similar_users_df.index.isin(items_notbuy)]
    # order the dataframe
    similar_users_df_ordered = similar_users_df.sort_values(by=['mean'], ascending=False)
    # grab the top n item   
    top_n_items = similar_users_df_ordered.head(items)
    top_n_items_indices = top_n_items.index.tolist()
    
    # lookup these item in the other dataframe to find names
    #item_information = article[article['article_id'].isin(top_n_items_indices)]
    
    return top_n_items_indices

In [94]:
items = []
for i in  tqdm(range(0,10)):
    recommend = recommend_item(newdf['article_id_last'].iloc[i,],newdf['similarid'].iloc[i,], df)
    items.append(recommend)
recdf = pd.DataFrame(list(zip(df.index, items)),columns =['ID', 'pred'])
recdf.head()


100%|██████████| 10/10 [00:01<00:00,  8.18it/s]


,ID,pred
0,86,"[873217004, 822946001, 713253003, 754238024, 8..."
1,173,"[800691007, 706016001, 800691008, 717490017, 6..."
2,733,"[628425001, 864316001, 741356002, 879436002, 8..."
3,794,"[803468002, 789028003, 758034001, 859744001, 3..."
4,956,"[706016002, 706016001, 679853021, 706016003, 8..."
